In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 불러오기

In [2]:
# 모듈 불러오기

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

from os.path import join
import re

## 단어장 불러오기

In [164]:
base_dir = '/content/drive/MyDrive/T_academy/PROJECTS/StartupChallenge_Zeroxflow'

df_word_path = join(base_dir, 'prepro_data', '1003_preprocessed_words.csv')
df_word = pd.read_csv(df_word_path, encoding='utf-8')
print(df_word.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15642 entries, 0 to 15641
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   eng            15642 non-null  object
 1   level          15642 non-null  object
 2   more_than_one  15642 non-null  bool  
dtypes: bool(1), object(2)
memory usage: 259.8+ KB
None


In [165]:
# 결측 칼럼 확인
df_word.isna().sum()

eng              0
level            0
more_than_one    0
dtype: int64

In [166]:
#단어장 내 레벨별 단어 수 확인
from collections import Counter
print('학년별 단어 수')
display( Counter(df_word.level) )
print('총 단어 수',len(df_word))

학년별 단어 수


Counter({'E': 699,
         'H1': 3403,
         'H2': 5130,
         'H3': 909,
         'M1': 2058,
         'M2': 1873,
         'M3': 1570})

총 단어 수 15642


## 텍스트 가져오기

In [167]:
df_text_path = join(base_dir, 'prepro_data', '1003_preprocessed_text.csv')
df_text = pd.read_csv(df_text_path, encoding='utf-8', index_col=0)
print(df_text.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38256 entries, 0 to 38255
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   order                    38256 non-null  int64 
 1   video_id                 38256 non-null  int64 
 2   title                    38256 non-null  object
 3   level                    38256 non-null  object
 4   rem_art_lemmatized_text  38256 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.8+ MB
None


In [168]:
df_text

,order,video_id,title,level,rem_art_lemmatized_text
0,1,26657,수능특강 1 - Gateway,H3,"['dear', 'mr', 'kayne', 'be', 'resident', 'of'..."
1,2,26657,수능특강 1 - Gateway,H3,"['live', 'with', 'three', 'dog', 'be', 'very',..."
2,3,26657,수능특강 1 - Gateway,H3,"['however', 'noise', 'of', 'barking', 'and', '..."
3,4,26657,수능특강 1 - Gateway,H3,"['many', 'of', 'my', 'apartment', 'neighbor', ..."
4,5,26657,수능특강 1 - Gateway,H3,"['want', 'immediate', 'action', 'to', 'solve',..."
...,...,...,...,...,...
38251,17,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"['you', 'just', 'need', 'to', 'add', 'some', '..."
38252,18,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"['be', 'curious', 'katie', 'ask', 'how', 'can'..."
38253,19,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"['sally', 'take', 'out', 'container', 'of', 'v..."
38254,20,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"['smile', 'she', 'add', 'it', ""'s"", 'not', 'th..."


In [ ]:
# [] 빈 리스트인 로우 있다.

df_text[df_text['rem_art_lemmatized_text'] == '[]']

In [ ]:
df_text.drop(index=[], inplace=True)

## str -> list

In [111]:
type(df_text['rem_art_lemmatized_text'][0])

str

In [112]:
type(df_word['eng'][0])

str

In [169]:
from ast import literal_eval

In [170]:
df_word['eng'] = df_word['eng'].apply(literal_eval)

In [115]:
type(df_word['eng'][0])

list

In [171]:
df_text['rem_art_lemmatized_text'] = df_text['rem_art_lemmatized_text'].apply(literal_eval)

In [117]:
type(df_text['rem_art_lemmatized_text'][0])

list

In [118]:
df_text['rem_art_lemmatized_text']

0        [dear, mr, kayne, be, resident, of, cansinghil...
1        [live, with, three, dog, be, very, happy, to, ...
2        [however, noise, of, barking, and, yell, from,...
3        [many, of, my, apartment, neighbor, also, seri...
4        [want, immediate, action, to, solve, this, urg...
                               ...                        
38251    [you, just, need, to, add, some, color, to, pl...
38252        [be, curious, katie, ask, how, can, do, that]
38253    [sally, take, out, container, of, vegetable, f...
38254    [smile, she, add, it, 's, not, that, hard, and...
38255    [katie, smile, know, that, now, she, could, ma...
Name: rem_art_lemmatized_text, Length: 38256, dtype: object

# level word count 컬럼 만들기

In [172]:
# 비상용으로 텍스트 컬럼 만들어 놓기
df_text['rem_art_lemmatized_text2'] = df_text['rem_art_lemmatized_text']

In [173]:
#level_list 선언
level_list = ['E','M1','M2','M3', 'H1', 'H2', 'H3']

In [174]:
# 새로 컬럼 만들어 주기
for lv in level_list:
    df_text[lv] = 0

In [175]:
df_text

,order,video_id,title,level,rem_art_lemmatized_text,rem_art_lemmatized_text2,E,M1,M2,M3,H1,H2,H3
0,1,26657,수능특강 1 - Gateway,H3,"[dear, mr, kayne, be, resident, of, cansinghil...","[dear, mr, kayne, be, resident, of, cansinghil...",0,0,0,0,0,0,0
1,2,26657,수능특강 1 - Gateway,H3,"[live, with, three, dog, be, very, happy, to, ...","[live, with, three, dog, be, very, happy, to, ...",0,0,0,0,0,0,0
2,3,26657,수능특강 1 - Gateway,H3,"[however, noise, of, barking, and, yell, from,...","[however, noise, of, barking, and, yell, from,...",0,0,0,0,0,0,0
3,4,26657,수능특강 1 - Gateway,H3,"[many, of, my, apartment, neighbor, also, seri...","[many, of, my, apartment, neighbor, also, seri...",0,0,0,0,0,0,0
4,5,26657,수능특강 1 - Gateway,H3,"[want, immediate, action, to, solve, this, urg...","[want, immediate, action, to, solve, this, urg...",0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38251,17,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[you, just, need, to, add, some, color, to, pl...","[you, just, need, to, add, some, color, to, pl...",0,0,0,0,0,0,0
38252,18,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[be, curious, katie, ask, how, can, do, that]","[be, curious, katie, ask, how, can, do, that]",0,0,0,0,0,0,0
38253,19,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[sally, take, out, container, of, vegetable, f...","[sally, take, out, container, of, vegetable, f...",0,0,0,0,0,0,0
38254,20,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[smile, she, add, it, 's, not, that, hard, and...","[smile, she, add, it, 's, not, that, hard, and...",0,0,0,0,0,0,0


## 카운트

In [176]:
word_list = df_word[['eng','level']]

In [177]:
import copy

In [178]:
# 같이 작업하던 함수 # 업데이트 1002
def count_word_and_words(word_list: pd.DataFrame, sentence: list) -> pd.Series:
    cham = dict(zip(level_list, [0, 0, 0, 0, 0, 0, 0]))
    for j in (word_list.itertuples()):
        words = j.eng
        if all( elem in sentence for elem in words):
            try:
                # sentence에서 제거하는 로직
                sentence_temp = copy.deepcopy(sentence)
                for elem in words:
                    # idx = sentence.index(elem)
                    # del sentence[idx]
                    # sentence.pop(idx)
                    sentence_temp.remove(elem)
                sentence = sentence_temp
                level = j.level
                cham[level] += 1
            except:
                print(elem, ':', words, '\n', sentence)
                pass
            

        else :
            pass
        
    cham_series = pd.Series(cham)

    return cham_series

words 에 2개 있는데, text에 1개 있을 경우 에러??

In [179]:
# 이게 에러의 원인 같다.
aa = ['a','a','a']
bb = ['a','a', 'a', 'a']
all( elem in aa for elem in bb)

True

In [180]:
df_text.iloc[:, 6:] = df_text['rem_art_lemmatized_text'].apply(lambda x: count_word_and_words(word_list=word_list, sentence=x))

all : ['all', 'in', 'all'] 
 ['she', 'love', 'nothing', 'well', 'than', 'to', 'have', 'all', 'of', 'her', 'child', 'in', 'house', 'share', 'fun', 'and', 'food']
arm : ['arm', 'in', 'arm'] 
 ['suddenly', 'he', 'be', 'stand', 'in', 'my', 'doorway', 'with', 'his', 'right', 'arm', 'raise']
again : ['again', 'and', 'again'] 
 ['people', 'nod', 'and', 'smile', 'brief', 'hello', 'and', 'and', 'then', 'disappear', 'into', 'dwell', 'and', 'we', 'would', "n't", 'see', 'them', 'again', 'day']
over : ['over', 'and', 'over'] 
 ['over', 'next', 'few', 'month', 'family', 'organize', 'neighborhood', 'and', 'open', 'house', 'brunch']
all : ['all', 'in', 'all'] 
 ['but', 'even', 'if', 'all', 'u', 'gain', 'emotional', 'and', 'intellectual', 'maturity', 'to', 'finally', 'divest', 'ourselves', 'from', 'harm', 'them', 'we', 'may', 'lose', 'them', 'in', 'ocean', 'of', 'plastic', 'chemical', 'fishing', 'tangle', 'spin', 'propeller', 'speed', 'hull', 'and', 'noise']
hand : ['hand', 'in', 'hand'] 
 ['there', 'n

In [181]:
df_text

,order,video_id,title,level,rem_art_lemmatized_text,rem_art_lemmatized_text2,E,M1,M2,M3,H1,H2,H3
0,1,26657,수능특강 1 - Gateway,H3,"[dear, mr, kayne, be, resident, of, cansinghil...","[dear, mr, kayne, be, resident, of, cansinghil...",5,2,0,3,1,2,0
1,2,26657,수능특강 1 - Gateway,H3,"[live, with, three, dog, be, very, happy, to, ...","[live, with, three, dog, be, very, happy, to, ...",5,5,1,0,2,2,0
2,3,26657,수능특강 1 - Gateway,H3,"[however, noise, of, barking, and, yell, from,...","[however, noise, of, barking, and, yell, from,...",6,3,0,3,1,5,1
3,4,26657,수능특강 1 - Gateway,H3,"[many, of, my, apartment, neighbor, also, seri...","[many, of, my, apartment, neighbor, also, seri...",4,1,0,2,0,4,0
4,5,26657,수능특강 1 - Gateway,H3,"[want, immediate, action, to, solve, this, urg...","[want, immediate, action, to, solve, this, urg...",4,3,0,0,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38251,17,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[you, just, need, to, add, some, color, to, pl...","[you, just, need, to, add, some, color, to, pl...",3,2,2,0,0,3,0
38252,18,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[be, curious, katie, ask, how, can, do, that]","[be, curious, katie, ask, how, can, do, that]",4,1,0,1,0,3,0
38253,19,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[sally, take, out, container, of, vegetable, f...","[sally, take, out, container, of, vegetable, f...",5,3,2,1,1,4,1
38254,20,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[smile, she, add, it, 's, not, that, hard, and...","[smile, she, add, it, 's, not, that, hard, and...",13,3,1,0,0,5,0


In [182]:
# savepoint

words_path = join(base_dir, 'prepro_data', '1003_sample_new.csv')
df_text.to_csv(words_path, index=False, encoding='utf-8')

# 피처 만들기

In [183]:
df_text2 = df_text.copy()

## 문제 별 문장의 수

In [185]:
df_text2['count_sentence_per_text'] = 1 # 얘를 groupby 하면서 집계하면 됨.
df_text2.head()

,order,video_id,title,level,rem_art_lemmatized_text,rem_art_lemmatized_text2,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text
0,1,26657,수능특강 1 - Gateway,H3,"[dear, mr, kayne, be, resident, of, cansinghil...","[dear, mr, kayne, be, resident, of, cansinghil...",5,2,0,3,1,2,0,1
1,2,26657,수능특강 1 - Gateway,H3,"[live, with, three, dog, be, very, happy, to, ...","[live, with, three, dog, be, very, happy, to, ...",5,5,1,0,2,2,0,1
2,3,26657,수능특강 1 - Gateway,H3,"[however, noise, of, barking, and, yell, from,...","[however, noise, of, barking, and, yell, from,...",6,3,0,3,1,5,1,1
3,4,26657,수능특강 1 - Gateway,H3,"[many, of, my, apartment, neighbor, also, seri...","[many, of, my, apartment, neighbor, also, seri...",4,1,0,2,0,4,0,1
4,5,26657,수능특강 1 - Gateway,H3,"[want, immediate, action, to, solve, this, urg...","[want, immediate, action, to, solve, this, urg...",4,3,0,0,1,3,0,1


## 문장의 단어 갯수

In [186]:
df_text2['mean_count_word_per_sentence'] = df_text2['rem_art_lemmatized_text2'].apply(lambda x: len(x))
df_text2.head()

,order,video_id,title,level,rem_art_lemmatized_text,rem_art_lemmatized_text2,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence
0,1,26657,수능특강 1 - Gateway,H3,"[dear, mr, kayne, be, resident, of, cansinghil...","[dear, mr, kayne, be, resident, of, cansinghil...",5,2,0,3,1,2,0,1,17
1,2,26657,수능특강 1 - Gateway,H3,"[live, with, three, dog, be, very, happy, to, ...","[live, with, three, dog, be, very, happy, to, ...",5,5,1,0,2,2,0,1,21
2,3,26657,수능특강 1 - Gateway,H3,"[however, noise, of, barking, and, yell, from,...","[however, noise, of, barking, and, yell, from,...",6,3,0,3,1,5,1,1,22
3,4,26657,수능특강 1 - Gateway,H3,"[many, of, my, apartment, neighbor, also, seri...","[many, of, my, apartment, neighbor, also, seri...",4,1,0,2,0,4,0,1,11
4,5,26657,수능특강 1 - Gateway,H3,"[want, immediate, action, to, solve, this, urg...","[want, immediate, action, to, solve, this, urg...",4,3,0,0,1,3,0,1,8


In [189]:
# 합계 알파벳 수
df_text2['count_word_per_text'] = df_text2['mean_count_word_per_sentence']

## 단어의 길이(문장의 알파벳 수)
- 합계 알파벳 수 / 합계 단어 수

In [187]:
df_text2['count_alphabet_per_word'] = df_text2['rem_art_lemmatized_text2'].apply(lambda x: len(''.join(x)))

In [188]:
df_text2.head()

,order,video_id,title,level,rem_art_lemmatized_text,rem_art_lemmatized_text2,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_sentence
0,1,26657,수능특강 1 - Gateway,H3,"[dear, mr, kayne, be, resident, of, cansinghil...","[dear, mr, kayne, be, resident, of, cansinghil...",5,2,0,3,1,2,0,1,17,82
1,2,26657,수능특강 1 - Gateway,H3,"[live, with, three, dog, be, very, happy, to, ...","[live, with, three, dog, be, very, happy, to, ...",5,5,1,0,2,2,0,1,21,87
2,3,26657,수능특강 1 - Gateway,H3,"[however, noise, of, barking, and, yell, from,...","[however, noise, of, barking, and, yell, from,...",6,3,0,3,1,5,1,1,22,89
3,4,26657,수능특강 1 - Gateway,H3,"[many, of, my, apartment, neighbor, also, seri...","[many, of, my, apartment, neighbor, also, seri...",4,1,0,2,0,4,0,1,11,60
4,5,26657,수능특강 1 - Gateway,H3,"[want, immediate, action, to, solve, this, urg...","[want, immediate, action, to, solve, this, urg...",4,3,0,0,1,3,0,1,8,43


## 최대 단어 길이

빈 리스트로 있던 애들 드랍

In [197]:
df_text2[df_text2['mean_count_word_per_sentence']==0].index

Int64Index([2862, 2868, 2873, 2877, 2882, 11101, 14287, 14293], dtype='int64')

In [198]:
df_text2 = df_text2.drop(index=[2862, 2868, 2873, 2877, 2882, 11101, 14287, 14293]).reset_index(drop=True)

In [199]:
df_text2['mean_logest_word_per_sentence'] = df_text2['rem_art_lemmatized_text2'].apply(lambda x: len(max(x, key=len)))
df_text2

,index,order,video_id,title,level,rem_art_lemmatized_text,rem_art_lemmatized_text2,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_sentence,count_word_per_text,mean_logest_word_per_sentence
0,0,1,26657,수능특강 1 - Gateway,H3,"[dear, mr, kayne, be, resident, of, cansinghil...","[dear, mr, kayne, be, resident, of, cansinghil...",5,2,0,3,1,2,0,1,17,82,17,11
1,1,2,26657,수능특강 1 - Gateway,H3,"[live, with, three, dog, be, very, happy, to, ...","[live, with, three, dog, be, very, happy, to, ...",5,5,1,0,2,2,0,1,21,87,21,12
2,2,3,26657,수능특강 1 - Gateway,H3,"[however, noise, of, barking, and, yell, from,...","[however, noise, of, barking, and, yell, from,...",6,3,0,3,1,5,1,1,22,89,22,9
3,3,4,26657,수능특강 1 - Gateway,H3,"[many, of, my, apartment, neighbor, also, seri...","[many, of, my, apartment, neighbor, also, seri...",4,1,0,2,0,4,0,1,11,60,11,9
4,4,5,26657,수능특강 1 - Gateway,H3,"[want, immediate, action, to, solve, this, urg...","[want, immediate, action, to, solve, this, urg...",4,3,0,0,1,3,0,1,8,43,8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38243,38251,17,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[you, just, need, to, add, some, color, to, pl...","[you, just, need, to, add, some, color, to, pl...",3,2,2,0,0,3,0,1,9,32,9,5
38244,38252,18,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[be, curious, katie, ask, how, can, do, that]","[be, curious, katie, ask, how, can, do, that]",4,1,0,1,0,3,0,1,8,29,8,7
38245,38253,19,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[sally, take, out, container, of, vegetable, f...","[sally, take, out, container, of, vegetable, f...",5,3,2,1,1,4,1,1,20,95,20,12
38246,38254,20,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[smile, she, add, it, 's, not, that, hard, and...","[smile, she, add, it, 's, not, that, hard, and...",13,3,1,0,0,5,0,1,25,101,25,9


## 문장 내 동사 갯수

## 문장 내 전치사 갯수

In [ ]:
import nltk
nltk.download('punkt')
nltk.download("averaged_perceptron_tagger")

In [202]:
# 한번에
count_verb = [ ]
count_proposition = [ ]


for i in df_text2['rem_art_lemmatized_text2']:
    npt = nltk.pos_tag(i)
    count_V = 0
    count_P = 0

    for j in npt :
        a = j[1] 
        if a.find("V") ==0 :
            count_V +=1
        elif a.find("IN") ==0 :
            count_P +=1
        elif a.find("TO") ==0 :
            count_P +=1
        else:
            continue
    count_verb.append(count_V)
    count_proposition.append(count_P)

In [204]:
df_text2['mean_count_verb_per_sentence'] = count_verb
df_text2['mean_count_proposition_per_sentence'] = count_proposition
df_text2

,index,order,video_id,title,level,rem_art_lemmatized_text,rem_art_lemmatized_text2,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_sentence,count_word_per_text,mean_logest_word_per_sentence,mean_count_verb_per_sentence,mean_count_proposition_per_sentence
0,0,1,26657,수능특강 1 - Gateway,H3,"[dear, mr, kayne, be, resident, of, cansinghil...","[dear, mr, kayne, be, resident, of, cansinghil...",5,2,0,3,1,2,0,1,17,82,17,11,2,2
1,1,2,26657,수능특강 1 - Gateway,H3,"[live, with, three, dog, be, very, happy, to, ...","[live, with, three, dog, be, very, happy, to, ...",5,5,1,0,2,2,0,1,21,87,21,12,4,4
2,2,3,26657,수능특강 1 - Gateway,H3,"[however, noise, of, barking, and, yell, from,...","[however, noise, of, barking, and, yell, from,...",6,3,0,3,1,5,1,1,22,89,22,9,2,4
3,3,4,26657,수능특강 1 - Gateway,H3,"[many, of, my, apartment, neighbor, also, seri...","[many, of, my, apartment, neighbor, also, seri...",4,1,0,2,0,4,0,1,11,60,11,9,1,2
4,4,5,26657,수능특강 1 - Gateway,H3,"[want, immediate, action, to, solve, this, urg...","[want, immediate, action, to, solve, this, urg...",4,3,0,0,1,3,0,1,8,43,8,9,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38243,38251,17,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[you, just, need, to, add, some, color, to, pl...","[you, just, need, to, add, some, color, to, pl...",3,2,2,0,0,3,0,1,9,32,9,5,3,2
38244,38252,18,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[be, curious, katie, ask, how, can, do, that]","[be, curious, katie, ask, how, can, do, that]",4,1,0,1,0,3,0,1,8,29,8,7,3,0
38245,38253,19,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[sally, take, out, container, of, vegetable, f...","[sally, take, out, container, of, vegetable, f...",5,3,2,1,1,4,1,1,20,95,20,12,4,4
38246,38254,20,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[smile, she, add, it, 's, not, that, hard, and...","[smile, she, add, it, 's, not, that, hard, and...",13,3,1,0,0,5,0,1,25,101,25,9,5,1


In [207]:
df_text2.drop(columns=['index'], inplace=True)

In [208]:
# savepoint

words_path = join(base_dir, 'prepro_data', '1004_sample_new2.csv')
df_text2.to_csv(words_path, index=False, encoding='utf-8')

In [209]:
df_text3 = df_text2.copy()

# groupby

In [210]:
df_text3

,order,video_id,title,level,rem_art_lemmatized_text,rem_art_lemmatized_text2,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_sentence,count_word_per_text,mean_logest_word_per_sentence,mean_count_verb_per_sentence,mean_count_proposition_per_sentence
0,1,26657,수능특강 1 - Gateway,H3,"[dear, mr, kayne, be, resident, of, cansinghil...","[dear, mr, kayne, be, resident, of, cansinghil...",5,2,0,3,1,2,0,1,17,82,17,11,2,2
1,2,26657,수능특강 1 - Gateway,H3,"[live, with, three, dog, be, very, happy, to, ...","[live, with, three, dog, be, very, happy, to, ...",5,5,1,0,2,2,0,1,21,87,21,12,4,4
2,3,26657,수능특강 1 - Gateway,H3,"[however, noise, of, barking, and, yell, from,...","[however, noise, of, barking, and, yell, from,...",6,3,0,3,1,5,1,1,22,89,22,9,2,4
3,4,26657,수능특강 1 - Gateway,H3,"[many, of, my, apartment, neighbor, also, seri...","[many, of, my, apartment, neighbor, also, seri...",4,1,0,2,0,4,0,1,11,60,11,9,1,2
4,5,26657,수능특강 1 - Gateway,H3,"[want, immediate, action, to, solve, this, urg...","[want, immediate, action, to, solve, this, urg...",4,3,0,0,1,3,0,1,8,43,8,9,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38243,17,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[you, just, need, to, add, some, color, to, pl...","[you, just, need, to, add, some, color, to, pl...",3,2,2,0,0,3,0,1,9,32,9,5,3,2
38244,18,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[be, curious, katie, ask, how, can, do, that]","[be, curious, katie, ask, how, can, do, that]",4,1,0,1,0,3,0,1,8,29,8,7,3,0
38245,19,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[sally, take, out, container, of, vegetable, f...","[sally, take, out, container, of, vegetable, f...",5,3,2,1,1,4,1,1,20,95,20,12,4,4
38246,20,233460,2021학년도 고3 9월 모의고사 43-45번,H3,"[smile, she, add, it, 's, not, that, hard, and...","[smile, she, add, it, 's, not, that, hard, and...",13,3,1,0,0,5,0,1,25,101,25,9,5,1


In [232]:
# sum
sample = df_text3.groupby('video_id').sum()
sample

,order,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_sentence,count_word_per_text,mean_logest_word_per_sentence,mean_count_verb_per_sentence,mean_count_proposition_per_sentence
video_id,,,,,,,,,,,,,,,
26657,28,35,16,4,9,8,20,1,7,103,454,103,61,14,20
26658,45,45,24,13,9,14,25,6,9,163,750,163,90,28,24
26659,36,59,17,7,10,14,22,0,8,139,609,139,81,27,24
26660,36,44,15,9,2,11,22,2,8,128,601,128,71,15,18
26661,55,54,7,14,8,12,22,5,10,123,580,123,91,24,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233456,28,33,20,31,15,20,20,1,7,162,871,162,83,25,31
233457,28,25,20,14,8,22,25,3,7,131,784,131,82,13,18
233458,55,44,15,12,13,20,43,1,10,168,942,168,123,29,33


In [215]:
sample2 = sample.copy(deep=True)

In [234]:
word_sum = (sample.iloc[:, 1:8].sum(axis=1))

In [235]:
# 단어 비율 # 단어 카운트 합산으로 나눈 버전

for i in range(1,8):
    sample.iloc[:, i] = sample.iloc[:, i] / word_sum
sample

,order,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_sentence,count_word_per_text,mean_logest_word_per_sentence,mean_count_verb_per_sentence,mean_count_proposition_per_sentence
video_id,,,,,,,,,,,,,,,
26657,28,0.376344,0.172043,0.043011,0.096774,0.086022,0.215054,0.010753,7,103,454,103,61,14,20
26658,45,0.330882,0.176471,0.095588,0.066176,0.102941,0.183824,0.044118,9,163,750,163,90,28,24
26659,36,0.457364,0.131783,0.054264,0.077519,0.108527,0.170543,0.000000,8,139,609,139,81,27,24
26660,36,0.419048,0.142857,0.085714,0.019048,0.104762,0.209524,0.019048,8,128,601,128,71,15,18
26661,55,0.442623,0.057377,0.114754,0.065574,0.098361,0.180328,0.040984,10,123,580,123,91,24,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233456,28,0.235714,0.142857,0.221429,0.107143,0.142857,0.142857,0.007143,7,162,871,162,83,25,31
233457,28,0.213675,0.170940,0.119658,0.068376,0.188034,0.213675,0.025641,7,131,784,131,82,13,18
233458,55,0.297297,0.101351,0.081081,0.087838,0.135135,0.290541,0.006757,10,168,942,168,123,29,33


In [236]:
sample.iloc[:, 1:8].mean()

E     0.372603
M1    0.163700
M2    0.075728
M3    0.068157
H1    0.090168
H2    0.216683
H3    0.012961
dtype: float64

In [237]:
# 단어 비율 # 총 리스트 길이로 나눈 버전

for i in range(1,8):
    sample2.iloc[:, i] = sample2.iloc[:, i] / sample2['count_word_per_text']
sample2

,order,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_sentence,count_word_per_text,mean_logest_word_per_sentence,mean_count_verb_per_sentence,mean_count_proposition_per_sentence
video_id,,,,,,,,,,,,,,,
26657,28,0.339806,0.155340,0.038835,0.087379,0.077670,0.194175,0.009709,7,103,454,103,61,14,20
26658,45,0.276074,0.147239,0.079755,0.055215,0.085890,0.153374,0.036810,9,163,750,163,90,28,24
26659,36,0.424460,0.122302,0.050360,0.071942,0.100719,0.158273,0.000000,8,139,609,139,81,27,24
26660,36,0.343750,0.117188,0.070312,0.015625,0.085938,0.171875,0.015625,8,128,601,128,71,15,18
26661,55,0.439024,0.056911,0.113821,0.065041,0.097561,0.178862,0.040650,10,123,580,123,91,24,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233456,28,0.203704,0.123457,0.191358,0.092593,0.123457,0.123457,0.006173,7,162,871,162,83,25,31
233457,28,0.190840,0.152672,0.106870,0.061069,0.167939,0.190840,0.022901,7,131,784,131,82,13,18
233458,55,0.261905,0.089286,0.071429,0.077381,0.119048,0.255952,0.005952,10,168,942,168,123,29,33


In [238]:
sample2.iloc[:, 1:8].mean()

E     0.358372
M1    0.154769
M2    0.070077
M3    0.062078
H1    0.082160
H2    0.204509
H3    0.011526
dtype: float64

In [242]:
# sample.rename(columns={'count_alphabet_per_sentence':'count_alphabet_per_word'}, inplace=True)

In [243]:
# 합계 알파벳 수 / 합계 단어 수  합계 단어 수 드랍? (일단 남기자)

sample['count_alphabet_per_word'] = sample['count_alphabet_per_word'] / sample['count_word_per_text']
sample2['count_alphabet_per_word'] = sample2['count_alphabet_per_word'] / sample2['count_word_per_text']

In [245]:
# mean 컬럼들 / count_sentence_per_text

mean_list = ['mean_count_word_per_sentence', 'mean_logest_word_per_sentence', 'mean_count_verb_per_sentence', 'mean_count_proposition_per_sentence']

for c in mean_list:
    sample[c] = sample[c] / sample['count_sentence_per_text']
    sample2[c] = sample2[c] / sample2['count_sentence_per_text']

In [246]:
sample

,order,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_word,count_word_per_text,mean_logest_word_per_sentence,mean_count_verb_per_sentence,mean_count_proposition_per_sentence
video_id,,,,,,,,,,,,,,,
26657,28,0.376344,0.172043,0.043011,0.096774,0.086022,0.215054,0.010753,7,14.714286,4.407767,103,8.714286,2.000000,2.857143
26658,45,0.330882,0.176471,0.095588,0.066176,0.102941,0.183824,0.044118,9,18.111111,4.601227,163,10.000000,3.111111,2.666667
26659,36,0.457364,0.131783,0.054264,0.077519,0.108527,0.170543,0.000000,8,17.375000,4.381295,139,10.125000,3.375000,3.000000
26660,36,0.419048,0.142857,0.085714,0.019048,0.104762,0.209524,0.019048,8,16.000000,4.695312,128,8.875000,1.875000,2.250000
26661,55,0.442623,0.057377,0.114754,0.065574,0.098361,0.180328,0.040984,10,12.300000,4.715447,123,9.100000,2.400000,1.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233456,28,0.235714,0.142857,0.221429,0.107143,0.142857,0.142857,0.007143,7,23.142857,5.376543,162,11.857143,3.571429,4.428571
233457,28,0.213675,0.170940,0.119658,0.068376,0.188034,0.213675,0.025641,7,18.714286,5.984733,131,11.714286,1.857143,2.571429
233458,55,0.297297,0.101351,0.081081,0.087838,0.135135,0.290541,0.006757,10,16.800000,5.607143,168,12.300000,2.900000,3.300000


In [249]:
# y data
y_data = df_text3.groupby('video_id')['level'].agg('first')

In [252]:
sample = sample.join(y_data, how='left')

In [254]:
sample2 = sample2.join(y_data, how='left')

In [253]:
sample

,order,E,M1,M2,M3,H1,H2,H3,count_sentence_per_text,mean_count_word_per_sentence,count_alphabet_per_word,count_word_per_text,mean_logest_word_per_sentence,mean_count_verb_per_sentence,mean_count_proposition_per_sentence,level
video_id,,,,,,,,,,,,,,,,
26657,28,0.376344,0.172043,0.043011,0.096774,0.086022,0.215054,0.010753,7,14.714286,4.407767,103,8.714286,2.000000,2.857143,H3
26658,45,0.330882,0.176471,0.095588,0.066176,0.102941,0.183824,0.044118,9,18.111111,4.601227,163,10.000000,3.111111,2.666667,H3
26659,36,0.457364,0.131783,0.054264,0.077519,0.108527,0.170543,0.000000,8,17.375000,4.381295,139,10.125000,3.375000,3.000000,H3
26660,36,0.419048,0.142857,0.085714,0.019048,0.104762,0.209524,0.019048,8,16.000000,4.695312,128,8.875000,1.875000,2.250000,H3
26661,55,0.442623,0.057377,0.114754,0.065574,0.098361,0.180328,0.040984,10,12.300000,4.715447,123,9.100000,2.400000,1.800000,H3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233456,28,0.235714,0.142857,0.221429,0.107143,0.142857,0.142857,0.007143,7,23.142857,5.376543,162,11.857143,3.571429,4.428571,H3
233457,28,0.213675,0.170940,0.119658,0.068376,0.188034,0.213675,0.025641,7,18.714286,5.984733,131,11.714286,1.857143,2.571429,H3
233458,55,0.297297,0.101351,0.081081,0.087838,0.135135,0.290541,0.006757,10,16.800000,5.607143,168,12.300000,2.900000,3.300000,H3


In [255]:
# savepoint

words_path = join(base_dir, 'prepro_data', '1004_sample_with_feature.csv')
sample.to_csv(words_path, index=False, encoding='utf-8')

words_path = join(base_dir, 'prepro_data', '1004_sample_with_feature2.csv')
sample2.to_csv(words_path, index=False, encoding='utf-8')